In [1]:
import glob
from MTPHandler.core import Plate
from MTPHandler.readers import read_spectramax
from EnzymePynetics.core import Estimator

# Oxidation of ABTS by SLAC

## SLAC kinetics pH 3 from 25 °C to 45 °C

## Read in plate reader data

In [2]:
plates = []
for path in sorted(glob.glob("data/raw_data/*")):
    plates.append(
        Plate.from_reader(read_spectramax, path=path, ph=3.0, time_unit="min")
    )

### Define species

In [3]:
for plate in plates:
    # Assign buffer concentration
    buffer = plate.add_reactant(
        id="s0",
        name="sodium acetate",
        constant=True,
        init_conc=50,
        unit="mM",
    )
    plate.assign_species(species=buffer, to="all", init_conc=50, conc_unit="mmol / l")

    # Assign substrate concentrations
    abts = plate.add_reactant(
        id="s1",
        name="ABTS",
        constant=False,
        inchi="InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/b19-17-,20-18+",
        smiles=r"CCn\1c2ccc(cc2s/c1=N/N=c\3/n(c4ccc(cc4s3)S(=O)(=O)O)CC)S(=O)(=O)O",
    )

    plate.assign_species(
        species=abts,
        to="rows",
        init_conc=[0, 5, 10, 15, 25, 50, 75, 100, 150, 200],
        conc_unit="umol / l",
        ids=["A", "B", "C", "D", "E", "F"],
    )

    slac = plate.add_protein(
        id="p0",
        name="Small laccase (SLAC)",
        constant=False,
        ecnumber="1.10.3.2",
        uniprotid="Q9XAL8",
        sequence="MDRRGFNRRVLLGGAAAATSLSIAPEVAGAAPAAKGITARTAPAGGEVRHLKMYAEKLADGQMGYGFEKGKASVPGPLIEVNEGDTLHIEFTNTMDVRASLHVHGLDYEISSDGTAMNKSDVEPGGTRTYTWRTHKPGRRDDGTWRPGSAGYWHYHDHVVGTEHGTGGIRNGLYGPVIVRRKGDVLPDATHTIVFNDMTINNRKPHTGPDFEATVGDRVEIVMITHGEYYHTFHMHGHRWADNRTGILTGPDDPSRVIDNKITGPADSFGFQIIAGEGVGAGAWMYHCHVQSHSDMGMVGLFLVKKPDGTIPGYEPHEHGGATAKSGESGEPTGGAAAHEHEH",
    )
    plate.assign_species(
        species=slac,
        to="rows",
        ids=["A", "B", "C"],
        init_conc=0.073938727,
        conc_unit="umol / l",
    )

    # Add unobserved species
    abts_oxidised = plate.add_reactant(
        id="s2",
        name="oxidised ABTS radical",
        constant=False,
        inchi="InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/q+1/p-2",
        smiles=r"CCN\1C2=C(C=C(C=C2)S(=O)(=O)[O-])S/C1=N\N=C\3/[N+](C4=C(S3)C=C(C=C4)S(=O)(=O)[O-])CCCCn1/c(=N/N=c/2\sc3c(n2CC)ccc(c3)S(=O)(=O)O)/sc4c1ccc(c4)S(=O)(=O)O",
    )
    # Assign product concentrations
    plate.assign_species(
        species=abts_oxidised,
        to="rows",
        ids=["A", "B", "C"],
        init_conc=0,
        conc_unit="umol / l",
    )

    oxygen = plate.add_reactant(
        id="s3", name="Oxygen", constant=False, inchi="InChI=1S/O", smiles="O=O"
    )
    water = plate.add_reactant(
        id="s4",
        name="Water",
        constant=False,
        inchi="InChI=1S/H2O/h1H2",
        smiles="[H]O[H]",
    )

Assigned sodium acetate to all wells.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row A.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row B.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row C.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row D.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row E.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row F.
Assigned Small laccase (SLAC) with concentrations of [0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727] umol / l to row A.
Assigned Small laccase (SLAC) with concentrations of [0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 

In [4]:
plate.visualize(wavelengths=340)

### Blank absorption data

In the next step, the contribution of buffer and enzyme is subtracted from the absorption data.

In [5]:
for plate in plates:
    plate.blank_species(species=buffer, wavelength=340)
    plate.blank_species(species=slac, wavelength=340)

Mean absorption of sodium acetate at 50.0 mmol / l: 0.03316666666666667
Blanked 60 wells containing sodium acetate.
Mean absorption of Small laccase (SLAC) at 0.073938727 umol / l: 0.0010575757575757537
Blanked 30 wells containing Small laccase (SLAC).
Mean absorption of sodium acetate at 50.0 mmol / l: 0.039357575757575765
Blanked 60 wells containing sodium acetate.
Mean absorption of Small laccase (SLAC) at 0.073938727 umol / l: -0.0004606060606060675
Blanked 30 wells containing Small laccase (SLAC).
Mean absorption of sodium acetate at 50.0 mmol / l: 0.039539393939393944
Blanked 60 wells containing sodium acetate.
Mean absorption of Small laccase (SLAC) at 0.073938727 umol / l: 0.00011818181818181399
Blanked 30 wells containing Small laccase (SLAC).
Mean absorption of sodium acetate at 50.0 mmol / l: 0.03548484848484849
Blanked 60 wells containing sodium acetate.
Mean absorption of Small laccase (SLAC) at 0.073938727 umol / l: 0.002133333333333331
Blanked 30 wells containing Small l

### Concentration calculation and creation of `EnzymeMLDocument` objects

In the next cell, the calibration data of each plate is detected and subjected to a `Calibrator`, to compare different calibration equations, describing the relationship between absorption and concentration af ABTS.  
Visualization and comparison of the fit statistics reviled, that the calibration range needs to be limited to 0-150 µM of ABTS, since for concentrations above 150 µM, the absorption is not linear anymore. In order to describe this non-linear relationship, not enough sampling data between 150 µM and 200 µM is available. Thus accurate concentration calculation would not be possible in that range.  
In consequence, an upper absorption limit of $E_{340} = 3.1$ was chosen, which excludes calibration measurements with ABTS concentrations above 150 µM.

In [6]:
enzymemls = []

for plate in plates:
    abts_calibration = plate.calibrate(
        species=abts,
        wavelength=340,
        cutoff=3.1,
        signal_type="Absorbance",
    )

    # Get predefined calibration models
    linear, quadratic, cubic = abts_calibration.models

    # Fit models
    abts_calibration.fit_models()
    abts_calibration.visualize()
    abts_standard = abts_calibration.save_model(linear)
    with open(
        f"data/calibrations/{abts_standard.name}_standard_{abts_standard.temperature}C.json",
        "w",
    ) as f:
        f.write(abts_standard.json())

    enzymemls.append(
        plate.to_enzymeml(
            name=f"SLAC kinetic assay",
            detected_reactant=abts,
            wavelength=340,
            reactant_standard=abts_standard,
        )
    )

,AIC,R squared,RMSD
Model Name,,,
linear,-157,0.9965,0.0525
quadratic,-155,0.9965,0.0525
cubic,-153,0.9966,0.0523


Found 27 catalyzed wells


,AIC,R squared,RMSD
Model Name,,,
cubic,-246,0.9999,0.0094
quadratic,-238,0.9998,0.0113
linear,-235,0.9998,0.0124


Found 27 catalyzed wells


,AIC,R squared,RMSD
Model Name,,,
linear,-156,0.9962,0.0538
quadratic,-154,0.9962,0.0537
cubic,-152,0.9962,0.0536


Found 27 catalyzed wells


,AIC,R squared,RMSD
Model Name,,,
cubic,-250,0.9999,0.0087
linear,-222,0.9997,0.0159
quadratic,-220,0.9997,0.0158


Found 27 catalyzed wells


,AIC,R squared,RMSD
Model Name,,,
linear,-170,0.9978,0.0415
quadratic,-168,0.9978,0.0414
cubic,-166,0.9978,0.0412


Found 27 catalyzed wells


Since calibration data only allows to calculate the concentration of ABTS up to 150 µM, measurements with an initial concentration of 200 µM ABTS are removed from the dataset.

In [7]:
# Remove measurement entries with initial ABTS concentration of 200 µM
for enzymeml in enzymemls:
    del enzymeml.measurements[-1]

## Parameter estimation

A EnzymePynetics `Estimator` is initialized for each `EnzymeMLDocument`, which was create from individual `Plate` objects.
As the first step, the following reaction is formulated, creating a `Reaction`, which is added to the `Estimator`.  
ABTS + O<sub>2</sub> &rarr; ABTS<sup>&middot;+</sup> + H<sub>2</sub>O  


In [8]:
estimators = []
for enzymeml in enzymemls:
    estimator, _ = Estimator.from_enzymeml(enzymeml, abts)

    estimator.add_reaction(
        id="r0",
        name="Oxidation",
        educt=[abts, oxygen],
        product=[abts_oxidised, water],
        catalyst=slac,
    )

    estimators.append(estimator)

In [9]:
for estimator in estimators:
    # Substrate rate laws
    michaelis = estimator.add_model(
        id="model1",
        name="michaelis menten",
        equation="substrate = -(substrate * catalyst * k_cat) / (K_M + substrate)",
    )

    competitive_product = estimator.add_model(
        id="model2",
        name="competitive product inhibition",
        equation="substrate = -(substrate * catalyst * k_cat) / (K_M * (1 + product / K_ic) + substrate)",
    )

    substrate_inhibition = estimator.add_model(
        id="model3",
        name="substrate inhibition",
        equation="substrate = -(k_cat * catalyst * substrate) / (K_M + ((1+(substrate/K_iu))*substrate))",
    )

    # Enzyme rate laws
    enzyme_inactivation = estimator.add_model(
        id="model4",
        name="enzyme inactivation",
        equation="catalyst = -k_ie * catalyst ",
    )

In [10]:
for estimator in estimators:
    estimator.fit_models()
    estimator.correlations()

Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:07<00:00,  1.19s/it]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-669,28.775 ± 1 %,25.128 ± 3 %,1.145 ± 4 %,9.858 ± 12 %,,0.056 ± 5 %
michaelis menten with enzyme inactivation,-554,28.132 ± 1 %,31.459 ± 2 %,0.894 ± 3 %,,,0.075 ± 4 %
substrate inhibition with enzyme inactivation,-452,38.865 ± 5 %,52.254 ± 7 %,0.744 ± 9 %,,375.000 ± 15 %,0.076 ± 4 %
competitive product inhibition,-425,26.441 ± 3 %,15.223 ± 10 %,1.737 ± 11 %,2.009 ± 15 %,,
michaelis menten,-147,18.853 ± 2 %,31.588 ± 5 %,0.597 ± 5 %,,,
substrate inhibition,-117,25.986 ± 5 %,52.623 ± 9 %,0.494 ± 10 %,,375.000 ± 12 %,


Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:07<00:00,  1.26s/it]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-523,29.668 ± 2 %,22.311 ± 6 %,1.330 ± 6 %,7.756 ± 18 %,,0.048 ± 8 %
michaelis menten with enzyme inactivation,-447,31.247 ± 2 %,32.259 ± 3 %,0.969 ± 3 %,,,0.072 ± 4 %
competitive product inhibition,-404,25.706 ± 2 %,9.149 ± 16 %,2.810 ± 16 %,1.248 ± 21 %,,
substrate inhibition with enzyme inactivation,-309,43.755 ± 2 %,55.156 ± 3 %,0.793 ± 4 %,,375.000 ± 0 %,0.073 ± 5 %
michaelis menten,-92,21.279 ± 2 %,32.419 ± 5 %,0.656 ± 5 %,,,
substrate inhibition,-45,29.745 ± 9 %,55.584 ± 14 %,0.535 ± 17 %,,375.000 ± 27 %,


Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:06<00:00,  1.11s/it]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-329,34.765 ± 2 %,25.522 ± 6 %,1.362 ± 6 %,5.553 ± 15 %,,0.029 ± 16 %
competitive product inhibition,-297,33.471 ± 3 %,21.279 ± 8 %,1.573 ± 8 %,2.849 ± 12 %,,
michaelis menten with enzyme inactivation,-204,33.772 ± 2 %,33.563 ± 4 %,1.006 ± 4 %,,,0.066 ± 6 %
substrate inhibition with enzyme inactivation,-133,47.056 ± 8 %,56.535 ± 11 %,0.832 ± 14 %,,375.000 ± 18 %,0.067 ± 7 %
michaelis menten,-2,23.686 ± 2 %,33.732 ± 6 %,0.702 ± 6 %,,,
substrate inhibition,36,32.952 ± 3 %,57.032 ± 6 %,0.578 ± 7 %,,375.000 ± 12 %,


Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:07<00:00,  1.24s/it]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-284,54.309 ± 4 %,40.547 ± 5 %,1.339 ± 7 %,10.062 ± 15 %,,0.044 ± 10 %
competitive product inhibition,-212,74.629 ± 9 %,44.999 ± 10 %,1.658 ± 14 %,3.573 ± 13 %,,
michaelis menten with enzyme inactivation,-176,41.881 ± 2 %,40.628 ± 4 %,1.031 ± 5 %,,,0.075 ± 6 %
substrate inhibition with enzyme inactivation,-90,58.792 ± 10 %,68.247 ± 13 %,0.861 ± 16 %,,375.000 ± 34 %,0.075 ± 6 %
michaelis menten,63,28.269 ± 2 %,40.996 ± 6 %,0.690 ± 6 %,,,
substrate inhibition,103,39.671 ± 14 %,69.022 ± 19 %,0.575 ± 24 %,,375.000 ± 52 %,


Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:06<00:00,  1.16s/it]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-228,50.701 ± 2 %,40.590 ± 4 %,1.249 ± 4 %,28.009 ± 30 %,,0.058 ± 9 %
michaelis menten with enzyme inactivation,-215,50.045 ± 2 %,43.144 ± 3 %,1.160 ± 4 %,,,0.075 ± 4 %
competitive product inhibition,-150,52.027 ± 4 %,32.596 ± 8 %,1.596 ± 9 %,4.300 ± 11 %,,
substrate inhibition with enzyme inactivation,-90,71.253 ± 8 %,73.855 ± 12 %,0.965 ± 14 %,,375.000 ± 4 %,0.075 ± 5 %
michaelis menten,109,33.865 ± 2 %,43.647 ± 6 %,0.776 ± 6 %,,,
substrate inhibition,155,48.222 ± 13 %,74.961 ± 19 %,0.643 ± 23 %,,375.000 ± 36 %,


In [11]:
for estimator in estimators:
    estimator.visualize()

In [12]:
for estimator in estimators:
    model_name = "michaelis menten with enzyme inactivation"
    system = estimator.get_reaction_system(model_name)

In [13]:
from plotly import graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

model_name = "michaelis menten with enzyme inactivation"

# fig = make_subplots(rows=1, cols=2, subplot_titles=["kcat vs Km", "inactivation vs. temperature"])
fig = go.Figure()

for estimator in estimators:
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")
            fig.add_trace(
                go.Scatter(
                    y=[k_cat],
                    x=[K_m],
                    name=reaction.temperature,
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color=[reaction.temperature],
                        colorscale="bluered",
                    ),
                    showlegend=True,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[k_cat_stderr],
                        arrayminus=[k_cat_stderr],
                        visible=True,
                        color="black",
                    ),
                    error_x=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[K_m_stderr],
                        arrayminus=[K_m_stderr],
                        visible=True,
                        color="black",
                        # colorscale=px.colors.diverging.balance,
                    ),
                    # row=1,
                    # col=1,
                )
            )

fig.update_layout(
    template="simple_white",
    legend=dict(title=f"Temperature / {reaction.temperature_unit}"),
    yaxis_title=f"k<sub>cat</sub> / min<sup>-1</sup>",
    xaxis_title="K<sub>M</sub> / µM",
)


fig.show()

## Initial rates vs. full time course

In [14]:
init_rate_estimators = []
for enzymeml in enzymemls:
    estimator, _ = Estimator.from_enzymeml(enzymeml, abts)
    print(estimator.temperature)

    estimator.add_reaction(
        id="r0", name="Oxidation", educt=abts, product=abts_oxidised, catalyst=slac
    )

    estimator.add_model(
        id="model1",
        name="michaelis menten",
        equation="substrate = -(substrate * catalyst * k_cat) / (K_M + substrate)",
    )

    estimator.add_model(
        id="model2",
        name="competitive product inhibition",
        equation="substrate = -(substrate * catalyst * k_cat) / (K_M * (1 + product / K_ic) + substrate)",
    )

    # Enzyme rate laws
    estimator.add_model(
        id="model4",
        name="enzyme inactivation",
        equation="catalyst = -k_ie * catalyst",
    )

    estimator.fit_models(max_time=3)

    init_rate_estimators.append(estimator)

25.0


Fitting competitive product inhibition with enzyme inactivation model: 100%|██████████| 4/4 [00:02<00:00,  1.35it/s]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition,-203,26.091 ± 4 %,15.312 ± 18 %,1.704 ± 18 %,3.712 ± 34 %,,
competitive product inhibition with enzyme inactivation,-201,27.111 ± 7 %,15.519 ± 18 %,1.747 ± 19 %,3.879 ± 36 %,,0.032 ± > 100 %
michaelis menten with enzyme inactivation,-185,26.356 ± 8 %,22.914 ± 11 %,1.150 ± 13 %,,,0.079 ± 72 %
michaelis menten,-185,23.811 ± 3 %,22.940 ± 11 %,1.038 ± 11 %,,,


30.0


Fitting competitive product inhibition with enzyme inactivation model: 100%|██████████| 4/4 [00:03<00:00,  1.20it/s]


...

Fitting competitive product inhibition with enzyme inactivation model: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-158,56.880 ± 11 %,34.737 ± 12 %,1.637 ± 16 %,10.822 ± 42 %,,0.118 ± 45 %
competitive product inhibition,-155,53.226 ± 12 %,35.443 ± 13 %,1.502 ± 18 %,8.052 ± 37 %,,
michaelis menten with enzyme inactivation,-150,46.372 ± 7 %,33.259 ± 10 %,1.394 ± 12 %,,,0.163 ± 33 %
michaelis menten,-142,37.821 ± 4 %,33.342 ± 11 %,1.134 ± 11 %,,,


45.0


Fitting competitive product inhibition with enzyme inactivation model: 100%|██████████| 4/4 [00:04<00:00,  1.03s/it]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-175,57.727 ± 5 %,36.261 ± 8 %,1.592 ± 10 %,27.017 ± 69 %,,0.141 ± 29 %
michaelis menten with enzyme inactivation,-175,57.534 ± 5 %,37.980 ± 7 %,1.515 ± 9 %,,,0.165 ± 23 %
competitive product inhibition,-166,50.108 ± 4 %,34.886 ± 10 %,1.436 ± 10 %,13.100 ± 39 %,,
michaelis menten,-158,46.898 ± 3 %,38.111 ± 8 %,1.231 ± 9 %,,,


In [15]:
fig = go.Figure()


# Add traces for all data points
for estimator_id, estimator in enumerate(estimators):
    model_name = "michaelis menten with enzyme inactivation"
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")
            kcat_km = k_cat / K_m
            stderr = ((k_cat_stderr / k_cat) ** 2 +
                      (K_m_stderr / K_m) ** 2) ** 0.5
            print(stderr)
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[kcat_km],
                    x=[reaction.temperature],
                    name=f"full time-course",
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[stderr],
                        arrayminus=[stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                    showlegend=showlegend,
                )
            )

# Add traces for initial rates parameters
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")

            print(k_cat_stderr, K_m_stderr)
            kcat_km = k_cat / K_m
            stderr = ((k_cat_stderr / k_cat) ** 2 +
                      (K_m_stderr / K_m) ** 2) ** 0.5
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[kcat_km],
                    x=[reaction.temperature],
                    name=f"partially-fixed parameters",
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    legendgroup="Initial rates",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[stderr],
                        arrayminus=[stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"k<sub>cat</sub> K<sub>M</sub><sup>-1</sup> / min<sup>-1</sup> µM<sup>-1</sup>",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)


config = {
    "toImageButtonOptions": {
        "format": "png",  # one of png, svg, jpeg, webp
        "filename": "custom_image",
        # "height": 500,
        # "width": 700,
        "scale": 8,  # Multiply title/legend/axis/canvas sizes by this factor
    }
}


fig.show(config=config)

0.027653197354393658
0.0300692791586707
0.044782438891636946
0.04503574659388577
0.035980336008259804
2.0801729247130765 2.4149800297892927
2.1075463625973625 1.9416390941765604
3.596919012009563 3.355598442216522
3.440813115137302 3.343022996455979
3.012411151321133 2.730292857711554


In [16]:
for estimator in init_rate_estimators:
    estimator.fit_models_fixed_params(
        fixed_params=["k_cat", "K_M"], model="michaelis menten"
    )

Fitting michaelis menten model: 100%|██████████| 4/4 [00:02<00:00,  1.66it/s]                                       


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-514,23.811 ± 8 %,22.940 ± 12 %,1.038 ± 15 %,12.891 ± 11 %,,0.035 ± 8 %
michaelis menten with enzyme inactivation,-398,23.811 ± 9 %,22.940 ± 11 %,1.038 ± 14 %,,,0.061 ± 1 %
competitive product inhibition,-386,23.811 ± 4 %,22.940 ± 12 %,1.038 ± 13 %,4.891 ± 2 %,,
michaelis menten,-185,,,,,,


Fitting michaelis menten model: 100%|██████████| 4/4 [00:02<00:00,  1.63it/s]                                       


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-481,27.073 ± 8 %,22.781 ± 18 %,1.188 ± 20 %,9.417 ± 9 %,,0.035 ± 6 %
competitive product inhibition,-330,27.073 ± 3 %,22.781 ± 17 %,1.188 ± 17 %,3.737 ± 2 %,,
michaelis menten with enzyme inactivation,-306,27.073 ± 8 %,22.781 ± 9 %,1.188 ± 12 %,,,0.064 ± 1 %
michaelis menten,-185,,,,,,


Fitting michaelis menten model: 100%|██████████| 4/4 [00:02<00:00,  1.44it/s]                                       


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-285,30.152 ± 11 %,25.888 ± 13 %,1.165 ± 17 %,8.123 ± 12 %,,0.018 ± 23 %
competitive product inhibition,-268,30.152 ± 5 %,25.888 ± 13 %,1.165 ± 14 %,5.272 ± 2 %,,
michaelis menten with enzyme inactivation,-162,30.152 ± 12 %,25.888 ± 13 %,1.165 ± 18 %,,,0.060 ± 2 %
michaelis menten,-123,,,,,,


Fitting michaelis menten model: 100%|██████████| 4/4 [00:01<00:00,  2.13it/s]                                       


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-192,37.821 ± 17 %,33.342 ± 13 %,1.134 ± 21 %,30.396 ± 19 %,,0.041 ± 12 %
michaelis menten with enzyme inactivation,-153,37.821 ± 9 %,33.342 ± 10 %,1.134 ± 14 %,,,0.068 ± 2 %
michaelis menten,-142,,,,,,
competitive product inhibition,-121,37.821 ± 17 %,33.342 ± 14 %,1.134 ± 22 %,10.472 ± 2 %,,


Fitting michaelis menten model: 100%|██████████| 4/4 [00:01<00:00,  2.27it/s]                                       


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-218,46.898 ± 7 %,38.111 ± 8 %,1.231 ± 10 %,34.010 ± 28 %,,0.052 ± 9 %
michaelis menten with enzyme inactivation,-202,46.898 ± 6 %,38.111 ± 7 %,1.231 ± 10 %,,,0.070 ± 1 %
michaelis menten,-158,,,,,,
competitive product inhibition,-132,46.898 ± 4 %,38.111 ± 9 %,1.231 ± 10 %,7.341 ± 2 %,,


In [17]:
init_rate_estimators[-1].visualize()

In [18]:
init_rate_estimators[-2].visualize()

In [19]:
fig = go.Figure()


# Add traces for all data points
for estimator_id, estimator in enumerate(estimators):
    model_name = "michaelis menten with enzyme inactivation"
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")
            kcat_km = k_cat / K_m
            stderr = ((k_cat_stderr / k_cat) ** 2 +
                      (K_m_stderr / K_m) ** 2) ** 0.5
            print(stderr)
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_cat],
                    x=[reaction.temperature],
                    name=f"full time-course",
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[k_cat_stderr],
                        arrayminus=[k_cat_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                    showlegend=showlegend,
                )
            )

# Add traces for initial rates parameters
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")
            kcat_km = k_cat / K_m
            stderr = ((k_cat_stderr / k_cat) ** 2 +
                      (K_m_stderr / K_m) ** 2) ** 0.5
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_cat],
                    x=[reaction.temperature],
                    name=f"partially-fixed parameters",
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    legendgroup="Initial rates",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[k_cat_stderr],
                        arrayminus=[k_cat_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"k<sub>cat</sub> / min<sup>-1</sup>",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)

config = {
    "toImageButtonOptions": {
        "format": "png",  # one of png, svg, jpeg, webp
        "filename": "custom_image",
        # "height": 500,
        # "width": 700,
        "scale": 8,  # Multiply title/legend/axis/canvas sizes by this factor
    }
}


fig.show(config=config)

0.027653197354393658
0.0300692791586707
0.044782438891636946
0.04503574659388577
0.035980336008259804


In [20]:
fig = go.Figure()


# Add traces for all data points
for estimator_id, estimator in enumerate(estimators):
    model_name = "michaelis menten with enzyme inactivation"
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")
            kcat_km = k_cat / K_m
            stderr = ((k_cat_stderr / k_cat) ** 2 +
                      (K_m_stderr / K_m) ** 2) ** 0.5
            print(stderr)
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[K_m],
                    x=[reaction.temperature],
                    name=f"full time-course",
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[K_m_stderr],
                        arrayminus=[K_m_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                    showlegend=showlegend,
                )
            )

# Add traces for initial rates parameters
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")

            print(k_cat_stderr, K_m_stderr)
            kcat_km = k_cat / K_m
            stderr = ((k_cat_stderr / k_cat) ** 2 +
                      (K_m_stderr / K_m) ** 2) ** 0.5
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[K_m],
                    x=[reaction.temperature],
                    name=f"partially-fixed parameters",
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    legendgroup="Initial rates",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[K_m_stderr],
                        arrayminus=[K_m_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"K<sub>M</sub> / µM",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)


config = {
    "toImageButtonOptions": {
        "format": "png",  # one of png, svg, jpeg, webp
        "filename": "custom_image",
        # "height": 500,
        # "width": 700,
        "scale": 8,  # Multiply title/legend/axis/canvas sizes by this factor
    }
}


fig.show(config=config)

0.027653197354393658
0.0300692791586707
0.044782438891636946
0.04503574659388577
0.035980336008259804
2.0801729247130765 2.4149800297892927
2.1075463625973625 1.9416390941765604
3.596919012009563 3.355598442216522
3.440813115137302 3.343022996455979
3.012411151321133 2.730292857711554


# temperatur plots 
kie vs temp 



In [21]:
fig = go.Figure()


# Add traces for all data points
for estimator_id, estimator in enumerate(estimators):
    model_name = "competitive product inhibition"
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "K_ic":
                    k_ie = parameter.value
                    k_ie_stderr = parameter.stdev
                    k_ie_unit = parameter.unit
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_ie],
                    x=[reaction.temperature],
                    name=f"full time-course<br>parameter estimation",
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[k_ie_stderr],
                        arrayminus=[k_ie_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )

# Add traces for initial rates parameters
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "K_ic":
                    k_ie = parameter.value
                    k_ie_stderr = parameter.stdev
                    k_ie_unit = parameter.unit
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_ie],
                    x=[reaction.temperature],
                    name=f"partially-fixed<br>parameter estimation",
                    legendgroup="partially-fixed parameters",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[k_ie_stderr],
                        arrayminus=[k_ie_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"K<sub>ic</sub> / µM",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)


config = {
    "toImageButtonOptions": {
        "format": "png",  # one of png, svg, jpeg, webp
        "filename": "custom_image",
        # "height": 500,
        # "width": 700,
        "scale": 8,  # Multiply title/legend/axis/canvas sizes by this factor
    }
}


fig.show(config=config)

In [22]:
import numpy as np

fig = go.Figure()


# Add traces for all data points
for estimator_id, estimator in enumerate(estimators):
    model_name = "michaelis menten with enzyme inactivation"
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("catalyst"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_ie":
                    k_ie = np.log(2) / parameter.value
                    k_ie_stderr = parameter.stdev
                    k_ie_unit = parameter.unit
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_ie],
                    x=[reaction.temperature],
                    name=f"full time-course<br>parameter estimation",
                    text=[f"{kcat_km_corr:.2f}"],
                    hovertemplate="k<sub>cat</sub>: %{y:.2f} ± %{error_y.array:.2f} min<sup>-1</sup>"
                    + "<br>K<sub>M</sub>: %{x:.2f} ± %{error_x.array:.2f} µM"
                    + "<br>Correlation: %{text}"
                    + f"<br>Temperature: {reaction.temperature:.0f} {reaction.temperature_unit}"
                    + "<extra></extra>",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[np.log(2) / (k_ie - k_ie_stderr)],
                        arrayminus=[np.log(2) / (k_ie - k_ie_stderr)],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )

# Add traces for initial rates parameters
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("catalyst"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_ie":
                    k_ie = np.log(2) / parameter.value
                    k_ie_stderr = parameter.stdev
                    k_ie_unit = parameter.unit
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_ie],
                    x=[reaction.temperature],
                    name=f"partially-fixed<br>parameter estimation",
                    legendgroup="partially fixed parameters",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[np.log(2) / (k_ie - k_ie_stderr)],
                        arrayminus=[np.log(2) / (k_ie - k_ie_stderr)],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"half-life / min",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)


config = {
    "toImageButtonOptions": {
        "format": "png",  # one of png, svg, jpeg, webp
        "filename": "custom_image",
        # "height": 500,
        # "width": 700,
        "scale": 8,  # Multiply title/legend/axis/canvas sizes by this factor
    }
}


fig.show(config=config)

In [23]:
# Get the reaction system
result_enzymemls = []
for estimator, enzymeml in zip(init_rate_estimators, enzymemls):
    model_name = "michaelis menten with enzyme inactivation"
    reaction_system = estimator.get_reaction_system(model_name)

    # Save the reaction system to EnzymeML
    result_enzymemls.append(
        estimator.to_omex(
            enzymeml=enzymeml, reaction_system=reaction_system, out_path="data/enzymeml"
        )
    )


Archive was written to data/enzymeml/SLAC_kinetic_assay_25.0C.omex


Archive was written to data/enzymeml/SLAC_kinetic_assay_30.0C.omex


Archive was written to data/enzymeml/SLAC_kinetic_assay_35.5C.omex


Archive was written to data/enzymeml/SLAC_kinetic_assay_40.0C.omex


Archive was written to data/enzymeml/SLAC_kinetic_assay_45.0C.omex

